In [1]:
import sys
import numpy as np
from PyQt5.QtGui import *
from PyQt5.QtWidgets import *
from PyQt5.QtCore import *
import import_ipynb
from gdrawer import GDrawer
from Canvas import Canvas

import pickle

class MainWindow(QMainWindow):

    def __init__(self, parent = None ):
        QMainWindow.__init__(self, parent )

        self.thumbnail_size = 150
        self.resize(600, 500)

        bar = self.menuBar()
        fileMenu = bar.addMenu("File")
        actOpen = fileMenu.addAction( QIcon("./icons/open.png"), "&Open...", self.open, QKeySequence("Ctrl+O") )
        actSave = fileMenu.addAction( QIcon("./icons/save.png"), "&Save...", self.save, QKeySequence("Ctrl+S") )
        actQuit = fileMenu.addAction( QIcon("./icons/quit.png"), "&Quit...", self.quit, QKeySequence("Ctrl+Q") )

        fileToolBar = QToolBar("File")
        self.addToolBar( fileToolBar )
        fileToolBar.addAction( actOpen )
        fileToolBar.addAction( actSave )


        self.container = QWidget(self)
        v_layout = QVBoxLayout( self.container )
        self.container.setLayout( v_layout )


        ################################
        # TODO 2: create the template gallery
        ###############################
        self.gallery = self.create_template_gallery()
        v_layout.addWidget( self.gallery )

        self.canvas = Canvas()
        v_layout.addWidget(self.canvas)

        self.textEdit = QTextEdit( self )
        v_layout.addWidget( self.textEdit )

        self.setCentralWidget( self.container )

        ################################
        # TODO 9: connect the signal and the slot
        ###############################

        self.canvas.selected_template.connect(self.set_action_on_gesture)


        ################################
        # TODO 3: fill the template gallery
        ###############################
        name = ["Triangle", "X", "Rectangle", "Circle", "Check", "Caret", "Question", "Arrow", "left square bracket",
                "Right square bracket", "V", "Delete", "Left curly brace", "Right curly brace", "Star", "Pigtail"]
        #todo load the database
        d = pickle.load(open('./onedol_ds.pkl','rb'))
        data = d['dataset']
        labels = d['labels']
        all_gesture = False
        label = ''
        for gesture, label_index in zip(data, labels):
            if label_index != label:
                # todo 3 add the template in the gallery
                self.add_template_thumbnail(gesture,name[label_index])
                # todo 4 add the template to the one_dollar_recognizer
                self.canvas.oneDollar.addTemplate(gesture,name[label_index])
                if not all_gesture:
                    label = label_index


    ##########################
    # TODO 2: create the gallery (QListWidget)
    ##########################
    def create_template_gallery(self):
        gallery = QListWidget()
        gallery.setFixedHeight(150)
        gallery.setViewMode(True)
        gallery.setUniformItemSizes(True)
        size = QSize(60, 60);
        gallery.setIconSize(size)


        #todo 2 customize the gallery

        return gallery


    ################################
    # TODO 3: fill the template gallery
    ###############################
    def add_template_thumbnail(self, g, label):

        #draw the template path into a QIcon (icon)
        thumbnail_widget = GDrawer()
        thumbnail_widget.set_gesture_path(g, label )
        pix = QPixmap(thumbnail_widget.size())
        thumbnail_widget.render(pix, QPoint(), QRegion(0, 0, thumbnail_widget.width(), thumbnail_widget.height()));
        icon = QIcon(pix)

        #todo 3 create and add the corresponding item in the gallery
        item = QListWidgetItem(icon,label)
        self.gallery.addItem(item)



    #######################
    # TODO 9
    #######################
    def set_action_on_gesture(self, label, id, score):
        message = "template: " + label + " score: " + str(score)
        self.textEdit.setPlainText( message+  "\n" + self.textEdit.toPlainText() )

        #todo 9 select the corresponding element in the galery
        self.gallery.setCurrentRow(id)

    ##############
    def open(self):
        fileName = QFileDialog.getOpenFileName(self, "Open file", ".")
        file = open(fileName[0], 'r')
        str = file.readlines()
        str = '\n'.join(str)
        self.textEdit.setHtml( str )
        file.close()


    ###############
    def save(self):
        fileName = QFileDialog.getSaveFileName(self, "Save file", ".")
        with open(fileName[0], 'w') as fileSave:
            string = self.textEdit.toPlainText()
            fileSave.write(string)
            fileSave.close()
            print("Save... ", fileName[0], " saved.")

    ###############
    def quit(self):
        box = QMessageBox()
        b = box.question(self, 'Exit?', "Do you really want to exit ?", QMessageBox.Yes | QMessageBox.No)
        box.setIcon(QMessageBox.Question)
        if b == QMessageBox.Yes:
            sys.exit()

    def closeEvent(self, event):
        #event.ignore()
        #self.quit()
        return

    def log_action(self, str):
        content = self.textEdit.toPlainText()
        self.textEdit.setPlainText( content + "\n" + str)


importing Jupyter notebook from gdrawer.ipynb
importing Jupyter notebook from Canvas.ipynb
importing Jupyter notebook from onedollar.ipynb


In [ ]:
if __name__=="__main__":
    app = QApplication(sys.argv)

    window = MainWindow()
    window.show()
    app.exec_()


template id:  10  label:  V  score:  0.8976715604391774
template id:  8  label:  left square bracket  score:  0.8831752534342407
